### Automated Event Planning

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from litellm import completion


ollama_llm = completion(
            model="ollama/llama3.2:latest",
            api_base="http://localhost:11434"
            )

In [4]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/tharhtet/.local/share/virtualenvs/Agentic_RAG_projs-hQf1ctZM/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


#### Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",

    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=ollama_llm,
    allow_delegation=True
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=ollama_llm,
    allow_delegation=False
)

### Creating Venue Pydantic Object
Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator,
   
)

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    #async_execution=True,
    agent=logistics_manager,
  
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,

)

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
           
    
    verbose=True
)

#### Define Input


In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2023-05-12",
    'expected_participants': 200,
    'budget': 2000000,
    'venue_type': "Conference Hall"
}

In [14]:

result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3873975f-cd4a-4fa2-95e9-3d429f77d02e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 7695b680-4fa2-4abd-8d02-0d46e35707d3
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 7695b680-4fa2-4abd-8d02-0d46e35707d3
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Marketing and Communications Agent
## Final Answer: 
Based on the search results, I can't find any information about potential attendees for the Tech Innovation Conference in 2021. However, I found some relevant information about other tech conferences that might be of interest:

* The Future of Work Innovation Conference is happening in London in October 2025 and brings together people who are transforming the enterprise.
* The Gartner Tech Growth & Innovation Conference 2021 took place in the Americas and covered topics like accelerating growth in a new era.
* The CES (Consumer Electronics Show) is one of the largest technology-focused trade conferences in the world, happening from January 7-10, 2025.
* The RSA Conference 2025 will take place from April 28 to May 1, 2025.
* The TechInnovation conference happened virtually on September 28-30, 2021.

If you're looking for information about potential attendees for the Tech Innovation Conference in 2021, I would recommend sear

🚀 Crew: crew
├── 📋 Task: 7695b680-4fa2-4abd-8d02-0d46e35707d3
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
├── 📋 Task: ed69964b-98b7-44c7-b319-aa7213e64112
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Logistics Manager
│   │       Status: In Progress
│   ├── 🤖 Agent: Venue Coordinator
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
└── 📋 Task: dce98ef9-d7d5-4251-a985-44e1c0063b1f
       Status: Executing Task...
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 7695b680-4fa2-4abd-8d02-0d46e35707d3
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
├── 📋 Task: ed69964b-98b7-44c7-b319-aa7213e64112
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Logistics Manager
│   │       Status: In Progress
│   ├── 🤖 Agent: Venue Coordinator
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
└── 📋 Task: dce98ef9-d7d5-4251-a985-44e1c0063b1f
       Assigned to: Marketing and Communications Agent
       Status: ✅ Completed
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dce98ef9-d7d5-4251-a985-44e1c0063b1f                                                                     │
│  Agent: Marketing and Communications Agent                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3873975f-cd4a-4fa2-95e9-3d429f77d02e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [17]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Lancaster, PA',
 'booking_status': '',
 'capacity': 0,
 'name': 'Lancaster County Convention Center'}
